In [1]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout, IntProgress, Label
import os
import logging

In [3]:
import sys
sys.path.append(os.path.join(os.getcwd(), 'lib'))

In [4]:
from project_description import project_description
from settings import settings
from fragmax_logo import page_header
from log import OutputWidgetHandler
from db_objects import db_objects

ModuleNotFoundError: No module named 'settings'

In [ ]:
lp3_project_folder = os.path.join('/Users/tobkro/tmp/jupyter_test')
db_template = os.path.join(os.getcwd(),'database', 'fragmax.sqlite')

In [ ]:
logger = logging.getLogger(__name__)
handler = OutputWidgetHandler()
handler.setFormatter(logging.Formatter('%(asctime)s - [%(levelname)s] %(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)
logger.info('starting session')

In [ ]:
settingsObject = settings()
dbObject = db_objects()
# https://stackoverflow.com/questions/7501706/how-do-i-pass-variables-between-class-instances-or-get-the-caller
projectObject = project_description(settingsObject, dbObject, logger, lp3_project_folder, db_template)
headerObject = page_header()

In [ ]:
tab0 = VBox(children=[projectObject.grid_widget])
tab = widgets.Tab(children=[tab0])
tab.set_title(0, 'Project Description')
VBox(children=[headerObject.logo, tab])
# https://stackoverflow.com/questions/55763382/get-ipywidget-button-within-a-class-to-access-class-parameters-through-self

In [ ]:
handler.show_logs()